In [ ]:
#@markdown # **3. Install Dependencies (Required)**
#@markdown ---
#@markdown - It takes about ~8-10 minutes to install the required dependencies
#@markdown - In case you think it's not working, don't worry. Sit back and relax 😉

import warnings
warnings.filterwarnings("ignore")

%cd /content/gdrive/MyDrive/InvokeAI

# # Add the directory of your cloned repository to sys.path
import sys
sys.path.append('/content/gdrive/MyDrive/InvokeAI')

print('[1;32mInstalling dependencies...')

# Installing requirements from Dependencies folder
!pip install --upgrade pip setuptools > nul 2>&1
!pip install --use-pep517 --upgrade --force-reinstall -r Dependencies/Requirements.txt > nul 2>&1
!pip install -q -e . > nul 2>&1

print ('\u2714 Done')

/content/gdrive/MyDrive/InvokeAI
Installing dependencies...
✔ Done


In [ ]:
#@markdown # Option 1: Starting the Web UI with ngrok
!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

Ngrok_token = "2S8EGiMIUPE1Qj3fNmhCJUlxqZO_6G59BkdiDcyKM1oXtwxfA" #@param {type:"string"}
#@markdown - Add ngrok token (obtainable from https://ngrok.com)

#@markdown Only works with InvokeAI 3.0.2 and later

share=''
if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(9090 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True).public_url
  print(srv)
  get_ipython().system("invokeai-web  --root /content/gdrive/MyDrive/InvokeAI/GUI")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

In [ ]:
#@markdown # **5. Run InvokeAI**
#@markdown - Launching InvokeAI Web UI via SSH Tunnel
#@markdown - The outputs will be saved in the `Outputs` folder under `/content/gdrive/MyDrive/InvokeAI-v3.6.2/GUI` path
#@markdown - Open this first link ending with `.moe` to access the InvokeAI WebUI, once you see the this kind of url `http://127.0.0.1:9090` at the bottom
#@markdown ---

import os
import shlex
import subprocess
from IPython.display import clear_output
from pathlib import Path
from typing import Union
clear_output()


import warnings
warnings.filterwarnings("ignore")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

id_rsa_file = "/root/.ssh/id_rsa"
id_rsa_pub_file = "/root/.ssh/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)
clear_output()

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path("/root/.ssh/") / ssh_name
gen_key(ssh_path)
clear_output()

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/gdrive/MyDrive/InvokeAI/
!python /content/gdrive/MyDrive/InvokeAI/scripts/invokeai-web.py --root /content/gdrive/MyDrive/InvokeAI/GUI